### Stanford Dog Breed 데이터 세트를 아래 URL에서 직접 Download 및 압축 해제
* Kaggle의 Dataset으로 Object Storage 연결 시 이미지를 한장 씩 읽는 데 많은 시간이 소요되어 모델 학습에 시간이 더 걸림.
* Local Disk에 바로 이미지를 다운로드/압축 해제 후 모델에서 이를 이용할 수 있도록 함.

In [1]:
# stanford dog breed 데이터 세트 다운로드
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
# 현재 디렉토리인 /kaggle/working에 바로 압축 해제
!ls; tar -xvf images.tar

--2025-01-30 06:53:53--  http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 793579520 (757M) [application/x-tar]
Saving to: ‘images.tar’

images.tar          100%[===================>] 756.82M  32.4MB/s    in 16s     

2025-01-30 06:54:09 (48.7 MB/s) - ‘images.tar’ saved [793579520/793579520]

images.tar
Images/
Images/n02085620-Chihuahua/
Images/n02085620-Chihuahua/n02085620_10621.jpg
Images/n02085620-Chihuahua/n02085620_10976.jpg
Images/n02085620-Chihuahua/n02085620_11238.jpg
Images/n02085620-Chihuahua/n02085620_11258.jpg
Images/n02085620-Chihuahua/n02085620_11337.jpg
Images/n02085620-Chihuahua/n02085620_1152.jpg
Images/n02085620-Chihuahua/n02085620_11696.jpg
Images/n02085620-Chihuahua/n02085620_11818.jpg
Images/n02085620-Chihuahua/n02085620_11948.jpg
Images/n02085620-

### 이미지 파일들의 디렉토리와 파일명을 기반으로 메타 정보인 이미지 절대경로, 레이블을 DataFrame으로 생성
* /kaggle/working/Images 디렉토리 밑에 Dog breed 서브 디렉토리와 이미지 파일로 구성 되어 있음.
* 레이블 값은 이미지 파일의 절대경로에서 이미지 파일 바로 위에 있는 서브 디렉토리를 가공하여 생성.

In [2]:
import pandas as pd
import numpy as np
import os

IMAGE_DIR = '/kaggle/working/Images'

def make_dogbreed_dataframe(image_dir=IMAGE_DIR):
    paths = []
    label_gubuns = []
    for dirname, _, filenames in os.walk(image_dir):
        for filename in filenames:
            # 이미지 파일이 아닌 파일도 해당 디렉토리에 있음.
            if '.jpg' in filename:
                # 파일의 절대 경로를 file_path 변수에 할당.
                file_path = dirname+'/'+ filename
                paths.append(file_path)
                # 이미지 파일의 절대 경로에서 레이블명 생성을 위한 1차 추출. '/'로 분할하여 파일 바로 위 서브디렉토리 이름 가져옴.
                start_pos = file_path.find('/', 20)
                end_pos = file_path.rfind('/')
                imsi_breed = file_path[start_pos+1:end_pos]
                # 1차 추출된 데이터를 기반으로 '-' 이후 데이터가 레이블 값임.
                breed = imsi_breed[imsi_breed.find('-')+1:]
                #print(start_pos, end_pos, imsi_breed)
                label_gubuns.append(breed)

    data_df = pd.DataFrame({'path':paths, 'label':label_gubuns})

    #label에 따른 숫자 target 값 매핑
    sorted_label = np.sort(data_df['label'].unique())
    label_mapping = {label: index for index, label in enumerate(sorted_label)}
    data_df['target'] = data_df['label'].map(label_mapping)

    return data_df


### 전체 DataFrame을 학습과 테스트용 DataFrame으로 분리. 학습 DataFrame은 다시 학습과 검증용으로 분리
* train_test_split()을 이용하여 전체의 40%를 테스트 데이터로 할당. stratify인자로 breed label별로 균등하게 할당 설정.

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd

pd.set_option('display.max_colwidth', 200)
data_df = make_dogbreed_dataframe()

# 전체 데이터의 60%를 학습, 40%를 테스트로 분리.
train_df, test_df = train_test_split(data_df, test_size=0.4, stratify=data_df['label'], random_state=2025)
# 다시 학습 데이터의 80%를 학습, 20%를 검증으로 분리
tr_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['target'], random_state=2025)
print(tr_df.shape, val_df.shape, test_df.shape)

(9878, 3) (2470, 3) (8232, 3)


### Custom Dataset와 DataLoader 생성
* augmentation은 light한 augmention 부터 시작하여 점차 강도를 높임

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import albumentations as A
import cv2

class BreedDataset(Dataset):
    # 이미지 파일리스트, 타겟 파일리스트, transforms 등 이미지와 타겟 데이터 가공에 필요한 인자들을 입력 받음
    def __init__(self, image_paths, targets=None, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform

    # 전체 건수를 반환
    def __len__(self):
        return len(self.image_paths)

    # idx로 지정된 하나의 image, label을 tensor 형태로 반환
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        # opencv로 이미지 파일 로딩
        image_np = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB).astype(np.float32)
        # 보통은 transform이 None이 되는 경우는 거의 없음(Tensor 변환이라도 있음)
        image = self.transform(image=image_np)['image']

        if self.targets is not None:
            # 개별 target값을 tensor로 변환.
            target = torch.tensor(self.targets[idx])
            return image, target
        # 테스트 데이터의 경우 targets가 입력 되지 않을 수 있으므로 이를 대비.
        else:
            return image

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.2 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
from torch.optim import Adam
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2

class CFG:
    batch_size = 32
    image_size = 240
    
CFG.batch_size = 32 # 16
CFG.image_size = 240

# Horizontal_flip
tr_transform_eff = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Resize(CFG.image_size, CFG.image_size, p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

def create_tr_val_loader(tr_df, val_df, tr_transform, val_transform):
    tr_dataset =BreedDataset(image_paths=tr_df['path'].to_list(),
                               targets=tr_df['target'].to_list(), transform=tr_transform)
    val_dataset = BreedDataset(image_paths=val_df['path'].to_list(),
                               targets=val_df['target'].to_list(), transform=val_transform)

    tr_loader = DataLoader(tr_dataset, batch_size = CFG.batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=4*CFG.batch_size, shuffle=False, num_workers=4, pin_memory=True)

    return tr_loader, val_loader

tr_loader, val_loader = create_tr_val_loader(tr_df=tr_df, val_df=val_df,
                                             tr_transform=tr_transform_eff, val_transform=val_transform)
images, labels = next(iter(tr_loader))
print(images.shape, labels.shape)

torch.Size([32, 3, 240, 240]) torch.Size([32])


### torchvision Model 생성.
* Resnet101과 Efficientnet 계열을 테스트 할 수 있도록 모델 생성 함수 생성.  

In [6]:
import torch
import torch.nn as nn
from torchvision import models

def create_tv_model(model_name, num_classes=1000):
    model = None
    if model_name == 'efficientnet_v2_s':
        model = models.efficientnet_v2_s(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1280, out_features=num_classes))
    elif model_name == 'efficientnet_b4':
        model = models.efficientnet_b4(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1792, out_features=num_classes))
    elif model_name == 'efficientnet_b1':
        model = models.efficientnet_b1(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1280, out_features=num_classes))
    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1280, out_features=num_classes))
    elif model_name == 'resnet101':
        model = models.resnet101(weights='DEFAULT')
        model.fc = nn.Linear(in_features=2048, out_features=num_classes)

    return model

eff_model = create_tv_model('efficientnet_b0', num_classes=120)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 81.1MB/s]


### Trainer로 모델 학습 - Efficient B1 모델로 Fine Tuning 모델 학습

In [7]:
# /kaggle/working/modular/v1 디렉토리에 utils.py 파일 다운로드
!rm -rf ./modular/v1
!mkdir -p ./modular/v1
!wget -O ./modular/v1/utils.py https://raw.githubusercontent.com/chulminkw/CNN_PG_Torch/main/modular/v1/utils.py?raw=true
!ls ./modular/v1

import sys

# 반드시 system path를 아래와 같이 잡아줘야 함.
sys.path.append('/kaggle/working')

#아래가 수행되는지 반드시 확인
from modular.v1.utils import Trainer, ModelCheckpoint, EarlyStopping

--2025-01-30 06:56:28--  https://raw.githubusercontent.com/chulminkw/CNN_PG_Torch/main/modular/v1/utils.py?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13594 (13K) [text/plain]
Saving to: ‘./modular/v1/utils.py’

./modular/v1/utils. 100%[===================>]  13.28K  --.-KB/s    in 0s      

2025-01-30 06:56:28 (65.5 MB/s) - ‘./modular/v1/utils.py’ saved [13594/13594]

utils.py


#### 첫번째 Fine tuning 학습 - 먼저 마지막 FC layer들의 학습 파라미터만 Learnable 설정한 상태에서 학습
* 첫번째 학습시는 마지막 fc layer만 제외하고, Feature Extractor Layer의 parameter들을 모두 학습 파라미터 freeze하도록 requires_grad를 False로 설정

In [8]:
eff_b1_model = create_tv_model('efficientnet_b1', num_classes=120)
print(eff_b1_model.classifier)

Downloading: "https://download.pytorch.org/models/efficientnet_b1-c27df63c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1-c27df63c.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 169MB/s]


Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=120, bias=True)
)


In [9]:
from torchinfo import summary

eff_b1_model = create_tv_model('efficientnet_b1', num_classes=120)

def freeze_feature_extractor(model):
    for name, param in model.named_parameters():
        if 'classifier' in name:
            param.requires_grad = True
        else:
            param.requires_grad = False
    return model

eff_b1_model = freeze_feature_extractor(eff_b1_model)

summary(model=eff_b1_model, input_size=(1, 3, 240, 240),
       col_names=['input_size', 'output_size', 'trainable'],
       row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape               Output Shape              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 240, 240]          [1, 120]                  Partial
├─Sequential (features)                                      [1, 3, 240, 240]          [1, 1280, 8, 8]           False
│    └─Conv2dNormActivation (0)                              [1, 3, 240, 240]          [1, 32, 120, 120]         False
│    │    └─Conv2d (0)                                       [1, 3, 240, 240]          [1, 32, 120, 120]         False
│    │    └─BatchNorm2d (1)                                  [1, 32, 120, 120]         [1, 32, 120, 120]         False
│    │    └─SiLU (2)                                         [1, 32, 120, 120]         [1, 32, 120, 120]         --
│    └─Sequential (1)                                        [1, 32, 120, 120]         [1, 16, 120, 120]         False
│    │    └─MBConv (0)                       

In [10]:
def train_breed(model, tr_transform, val_transform, learning_rate=1e-3, epochs=30):
    tr_loader, val_loader = create_tr_val_loader(tr_df=tr_df, val_df=val_df,
                                             tr_transform=tr_transform, val_transform=val_transform)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    optimizer = Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer=optimizer, mode='min', factor=0.2, patience=3, threshold=0.01, min_lr=1e-7)

    trainer = Trainer(model=model, loss_fn=loss_fn, optimizer=optimizer,
                   train_loader=tr_loader, val_loader=val_loader, scheduler=scheduler, callbacks=None,
                   device=device)
    history = trainer.fit(epochs)

    return trainer, history

def train_breed_with_ft(model, tr_transform, val_transform, first_lr=1e-4, second_lr=1e-5,
                        first_epochs=15, second_epochs=15, stop_at_first=False):
    
    model = freeze_feature_extractor(model)
    print("#### first train with classifier layer ####")
    trainer, history = train_breed(model, tr_transform, val_transform, learning_rate=first_lr, epochs=first_epochs)

    model = trainer.get_trained_model()

    for param in model.parameters():
        param.requires_grad = True

    if not stop_at_first:
        print("#### final train with all layers ####")
        trainer, history = train_breed(model, tr_transform, val_transform, 
                                       learning_rate=second_lr, epochs=second_epochs)

    return trainer, history

In [11]:
eff_b1_model = create_tv_model('efficientnet_b1', num_classes=120)
trainer, history = train_breed_with_ft(model=eff_b1_model, 
                                       tr_transform=tr_transform_eff, val_transform=val_transform,
                                       first_lr=1e-4, second_lr=1e-5,
                                       first_epochs=15, second_epochs=15, stop_at_first=False)

#### first train with classifier layer ####


Epoch 1 [Validating]: 100%|██████████| 20/20 [00:09<00:00,  2.03it/s, Loss=4.42, Accuracy=0.486]


Epoch 1/15, Train Loss: 4.6127 Train Accuracy: 0.1939 , Val Loss: 4.4169 Val Accuracy: 0.4858 , Current lr:0.000100


Epoch 2 [Validating]: 100%|██████████| 20/20 [00:10<00:00,  1.95it/s, Loss=4.06, Accuracy=0.614]


Epoch 2/15, Train Loss: 4.2357 Train Accuracy: 0.5282 , Val Loss: 4.0607 Val Accuracy: 0.6138 , Current lr:0.000100


...

Epoch 13/15, Train Loss: 0.4080 Train Accuracy: 0.8808 , Val Loss: 0.4241 Val Accuracy: 0.8555 , Current lr:0.000010


Epoch 14 [Validating]: 100%|██████████| 20/20 [00:09<00:00,  2.07it/s, Loss=0.416, Accuracy=0.861]


Epoch 14/15, Train Loss: 0.3856 Train Accuracy: 0.8861 , Val Loss: 0.4165 Val Accuracy: 0.8607 , Current lr:0.000010


Epoch 15 [Validating]: 100%|██████████| 20/20 [00:09<00:00,  2.06it/s, Loss=0.418, Accuracy=0.856]

Epoch 15/15, Train Loss: 0.3733 Train Accuracy: 0.8935 , Val Loss: 0.4179 Val Accuracy: 0.8563 , Current lr:0.000010


In [12]:
#아래가 수행되는지 반드시 확인
from modular.v1.utils import Predictor

test_image_paths = test_df['path'].to_list()
test_targets = test_df['target'].to_list()

# 테스트 이미지 사이즈를 240으로 증가.
CFG.image_size = 240

test_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_dataset = BreedDataset(image_paths=test_image_paths,
                            targets=test_targets, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

trained_model = trainer.get_trained_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

[Evaluating]: 100%|██████████| 129/129 [00:34<00:00,  3.79it/s, Accuracy=0.859]

test dataset evaluation:0.8595


### EfficientNet B4 모델 학습 및 평가

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

def create_tv_model(model_name, num_classes=1000):
    model = None
    if model_name == 'efficientnet_v2_s':
        model = models.efficientnet_v2_s(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1280, out_features=num_classes))
    elif model_name == 'efficientnet_b4':
        model = models.efficientnet_b4(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1792, out_features=num_classes))
    elif model_name == 'efficientnet_b1':
        model = models.efficientnet_b1(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1280, out_features=num_classes))
    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights='DEFAULT')
        model.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                         nn.Linear(in_features=1280, out_features=num_classes))
    elif model_name == 'resnet101':
        model = models.resnet101(weights='DEFAULT')
        model.fc = nn.Linear(in_features=2048, out_features=num_classes)

    return model

In [15]:
from torchinfo import summary

eff_b4_model = models.efficientnet_b4(weights=None)

summary(model=eff_b4_model, input_size=(1, 3, 380, 380),
        col_names=['input_size', 'output_size', 'num_params'], 
        row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape               Output Shape              Param #
EfficientNet (EfficientNet)                                  [1, 3, 380, 380]          [1, 1000]                 --
├─Sequential (features)                                      [1, 3, 380, 380]          [1, 1792, 12, 12]         --
│    └─Conv2dNormActivation (0)                              [1, 3, 380, 380]          [1, 48, 190, 190]         --
│    │    └─Conv2d (0)                                       [1, 3, 380, 380]          [1, 48, 190, 190]         1,296
│    │    └─BatchNorm2d (1)                                  [1, 48, 190, 190]         [1, 48, 190, 190]         96
│    │    └─SiLU (2)                                         [1, 48, 190, 190]         [1, 48, 190, 190]         --
│    └─Sequential (1)                                        [1, 48, 190, 190]         [1, 24, 190, 190]         --
│    │    └─MBConv (0)                                       [1,

In [16]:
eff_b4_model = create_tv_model(model_name='efficientnet_b4', num_classes=120)
print(eff_b4_model.classifier)

Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:00<00:00, 91.4MB/s]


Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1792, out_features=120, bias=True)
)


In [17]:
models.EfficientNet_B4_Weights.IMAGENET1K_V1.transforms()

ImageClassification(
    crop_size=[380]
    resize_size=[384]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [18]:
CFG.batch_size = 16 # 메모리 부족 발생 시 Batch size를 16으로 낮춤. 
CFG.image_size = 380

# 학습용 Augmentation 적용. 
tr_transform_eff = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Resize(CFG.image_size, CFG.image_size, p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

#### B4 모델로 학습시 GPU 메모리 부족이 발생할 수 있음. 

In [19]:
# B4 모델은 학습 시간이 많이 걸리므로 실습을 위해 epochs를 20회 또는 10회로 수행. 
trainer, history = train_breed(model=eff_b4_model, tr_transform=tr_transform_eff, val_transform=val_transform,
                               learning_rate=1e-4, epochs=20)

Epoch 1 [Validating]: 100%|██████████| 39/39 [00:16<00:00,  2.32it/s, Loss=0.506, Accuracy=0.853]


Epoch 1/20, Train Loss: 2.4331 Train Accuracy: 0.4655 , Val Loss: 0.5057 Val Accuracy: 0.8530 , Current lr:0.000100


Epoch 2 [Validating]: 100%|██████████| 39/39 [00:16<00:00,  2.38it/s, Loss=0.366, Accuracy=0.88] 


Epoch 2/20, Train Loss: 0.7213 Train Accuracy: 0.7795 , Val Loss: 0.3660 Val Accuracy: 0.8802 , Current lr:0.000100


Epoch 3 [Validating]: 100%|██████████| 39/39 [00:16<00:00,  2.32it/s, Loss=0.309, Accuracy=0.896]


...

Epoch 18/20, Train Loss: 0.0638 Train Accuracy: 0.9829 , Val Loss: 0.3442 Val Accuracy: 0.9121 , Current lr:0.000001


Epoch 19 [Validating]: 100%|██████████| 39/39 [00:17<00:00,  2.27it/s, Loss=0.338, Accuracy=0.913]


Epoch 19/20, Train Loss: 0.0616 Train Accuracy: 0.9861 , Val Loss: 0.3376 Val Accuracy: 0.9130 , Current lr:0.000001


Epoch 20 [Validating]: 100%|██████████| 39/39 [00:17<00:00,  2.24it/s, Loss=0.341, Accuracy=0.909]

Epoch 20/20, Train Loss: 0.0609 Train Accuracy: 0.9862 , Val Loss: 0.3408 Val Accuracy: 0.9093 , Current lr:0.000001


In [20]:
#아래가 수행되는지 반드시 확인
from modular.v1.utils import Predictor

test_image_paths = test_df['path'].to_list()
test_targets = test_df['target'].to_list()

# 테스트 이미지 사이즈를 380으로 증가.
CFG.image_size = 380

test_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_dataset = BreedDataset(image_paths=test_image_paths,
                            targets=test_targets, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

trained_model = trainer.get_trained_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

[Evaluating]: 100%|██████████| 129/129 [00:54<00:00,  2.37it/s, Accuracy=0.908]

test dataset evaluation:0.9084


### EffcientNet-v2 Small 모델로 학습 및 평가
* 학습 이미지 사이즈를 384x384로 적용

In [24]:
from torchinfo import summary

eff_v2_s_model = models.efficientnet_v2_s(weights=None)

summary(model=eff_v2_s_model, input_size=(1, 3, 384, 384),
        col_names=['input_size', 'output_size', 'num_params'], 
        row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape               Output Shape              Param #
EfficientNet (EfficientNet)                                  [1, 3, 384, 384]          [1, 1000]                 --
├─Sequential (features)                                      [1, 3, 384, 384]          [1, 1280, 12, 12]         --
│    └─Conv2dNormActivation (0)                              [1, 3, 384, 384]          [1, 24, 192, 192]         --
│    │    └─Conv2d (0)                                       [1, 3, 384, 384]          [1, 24, 192, 192]         648
│    │    └─BatchNorm2d (1)                                  [1, 24, 192, 192]         [1, 24, 192, 192]         48
│    │    └─SiLU (2)                                         [1, 24, 192, 192]         [1, 24, 192, 192]         --
│    └─Sequential (1)                                        [1, 24, 192, 192]         [1, 24, 192, 192]         --
│    │    └─FusedMBConv (0)                                  [1, 2

In [25]:
eff_v2_s_model = create_tv_model(model_name='efficientnet_v2_s', num_classes=120)
print(eff_v2_s_model.classifier)

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=120, bias=True)
)


In [26]:
models.EfficientNet_V2_S_Weights.IMAGENET1K_V1.transforms()

ImageClassification(
    crop_size=[384]
    resize_size=[384]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [27]:
CFG.batch_size = 16 # 16
CFG.image_size = 384

# 학습용 Augmentation 적용. 
tr_transform_eff = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Resize(CFG.image_size, CFG.image_size, p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [28]:
# V2 모델은 학습 시간이 많이 걸리므로 실습을 위해 epochs를 20회 또는 10회로 수행. 
trainer, history = train_breed(model=eff_v2_s_model, tr_transform=tr_transform_eff, val_transform=val_transform,
                               learning_rate=1e-4, epochs=20)

Epoch 1 [Validating]: 100%|██████████| 39/39 [00:17<00:00,  2.28it/s, Loss=0.481, Accuracy=0.865]


Epoch 1/20, Train Loss: 1.7654 Train Accuracy: 0.6630 , Val Loss: 0.4810 Val Accuracy: 0.8648 , Current lr:0.000100


Epoch 2 [Validating]: 100%|██████████| 39/39 [00:16<00:00,  2.30it/s, Loss=0.445, Accuracy=0.869]


Epoch 2/20, Train Loss: 0.5122 Train Accuracy: 0.8609 , Val Loss: 0.4451 Val Accuracy: 0.8688 , Current lr:0.000100


Epoch 3 [Validating]: 100%|██████████| 39/39 [00:16<00:00,  2.37it/s, Loss=0.465, Accuracy=0.866]


...

Epoch 18/20, Train Loss: 0.0179 Train Accuracy: 0.9974 , Val Loss: 0.4384 Val Accuracy: 0.8923 , Current lr:0.000001


Epoch 19 [Validating]: 100%|██████████| 39/39 [00:16<00:00,  2.31it/s, Loss=0.454, Accuracy=0.894]


Epoch 19/20, Train Loss: 0.0169 Train Accuracy: 0.9973 , Val Loss: 0.4538 Val Accuracy: 0.8939 , Current lr:0.000000


Epoch 20 [Validating]: 100%|██████████| 39/39 [00:16<00:00,  2.31it/s, Loss=0.44, Accuracy=0.894] 

Epoch 20/20, Train Loss: 0.0165 Train Accuracy: 0.9970 , Val Loss: 0.4403 Val Accuracy: 0.8943 , Current lr:0.000000


In [30]:
#아래가 수행되는지 반드시 확인
from modular.v1.utils import Predictor

test_image_paths = test_df['path'].to_list()
test_targets = test_df['target'].to_list()

# 테스트 이미지 사이즈를 384로 증가.
CFG.image_size = 384

test_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_dataset = BreedDataset(image_paths=test_image_paths,
                            targets=test_targets, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

trained_model = trainer.get_trained_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

[Evaluating]: 100%|██████████| 129/129 [00:52<00:00,  2.46it/s, Accuracy=0.896]

test dataset evaluation:0.8957


### timm의 EfficientNet-v2 S 모델로 학습 및 평가
* efficientnetv2_rw_s.ra2_in1k는 상대적으로 학습 시간도 짧게 걸리고, 모델 성능도 비교적 뛰어남.
* 학습 시 사용된 이미지 사이즈는 288x288, 추론시에는 384x384 사용됨.
* tf_efficientnetv2_x 계열이나 다른 efficientnet 버전들은 생각보다 성능이 잘 나오지 않음.

In [36]:
import timm

timm.list_models(filter='*efficientnet*', pretrained=True)

['efficientnet_b0.ra4_e3600_r224_in1k',
 'efficientnet_b0.ra_in1k',
 'efficientnet_b1.ft_in1k',
 'efficientnet_b1.ra4_e3600_r240_in1k',
 'efficientnet_b1_pruned.in1k',
 'efficientnet_b2.ra_in1k',
 'efficientnet_b2_pruned.in1k',
 'efficientnet_b3.ra2_in1k',
 'efficientnet_b3_pruned.in1k',
 'efficientnet_b4.ra2_in1k',
 'efficientnet_b5.sw_in12k',
 'efficientnet_b5.sw_in12k_ft_in1k',
 'efficientnet_el.ra_in1k',
 'efficientnet_el_pruned.in1k',
 'efficientnet_em.ra2_in1k',
 'efficientnet_es.ra_in1k',
 'efficientnet_es_pruned.in1k',
 'efficientnet_lite0.ra_in1k',
 'efficientnetv2_rw_m.agc_in1k',
 'efficientnetv2_rw_s.ra2_in1k',
 'efficientnetv2_rw_t.ra2_in1k',
 'gc_efficientnetv2_rw_t.agc_in1k',
 'test_efficientnet.r160_in1k',
 'test_efficientnet_evos.r160_in1k',
 'test_efficientnet_gn.r160_in1k',
 'test_efficientnet_ln.r160_in1k',
 'tf_efficientnet_b0.aa_in1k',
 'tf_efficientnet_b0.ap_in1k',
 'tf_efficientnet_b0.in1k',
 'tf_efficientnet_b0.ns_jft_in1k',
 'tf_efficientnet_b1.aa_in1k',
 'tf_e

In [37]:
eff_v2_s_timm = timm.create_model(model_name='efficientnetv2_rw_s.ra2_in1k', pretrained=True, num_classes=120)
print(eff_v2_s_timm)

model.safetensors:   0%|          | 0.00/96.5M [00:00<?, ?B/s]

EfficientNet(
  (conv_stem): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): EdgeResidual(
        (conv_exp): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): Identity()
        (conv_pwl): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
        (drop_path): Identity()
      )
      (1): EdgeResidual(
        (conv_exp)

In [38]:
CFG.batch_size = 32 # 메모리 부족 발생 시 Batch size를 16으로 낮춤. 
CFG.image_size = 288 # timm model card에 학습시 288x288 적용

# 학습용 Augmentation 적용. 
tr_transform_eff = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Resize(CFG.image_size, CFG.image_size, p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [39]:
# epochs 20회만 수행. 
trainer, history = train_breed(model=eff_v2_s_timm, tr_transform=tr_transform_eff, val_transform=val_transform,
           learning_rate=1e-4, epochs=20)

Epoch 1 [Validating]: 100%|██████████| 20/20 [00:12<00:00,  1.56it/s, Loss=0.421, Accuracy=0.897]


Epoch 1/20, Train Loss: 2.0897 Train Accuracy: 0.5841 , Val Loss: 0.4209 Val Accuracy: 0.8968 , Current lr:0.000100


Epoch 2 [Validating]: 100%|██████████| 20/20 [00:12<00:00,  1.65it/s, Loss=0.353, Accuracy=0.892]


Epoch 2/20, Train Loss: 0.3579 Train Accuracy: 0.9017 , Val Loss: 0.3528 Val Accuracy: 0.8919 , Current lr:0.000100


Epoch 3 [Validating]: 100%|██████████| 20/20 [00:12<00:00,  1.61it/s, Loss=0.338, Accuracy=0.901]


...

Epoch 18/20, Train Loss: 0.0067 Train Accuracy: 0.9992 , Val Loss: 0.3711 Val Accuracy: 0.9081 , Current lr:0.000001


Epoch 19 [Validating]: 100%|██████████| 20/20 [00:12<00:00,  1.65it/s, Loss=0.371, Accuracy=0.906]


Epoch 19/20, Train Loss: 0.0069 Train Accuracy: 0.9991 , Val Loss: 0.3710 Val Accuracy: 0.9061 , Current lr:0.000000


Epoch 20 [Validating]: 100%|██████████| 20/20 [00:12<00:00,  1.58it/s, Loss=0.38, Accuracy=0.904] 

Epoch 20/20, Train Loss: 0.0071 Train Accuracy: 0.9989 , Val Loss: 0.3799 Val Accuracy: 0.9045 , Current lr:0.000000


In [40]:
#아래가 수행되는지 반드시 확인
from modular.v1.utils import Predictor

test_image_paths = test_df['path'].to_list()
test_targets = test_df['target'].to_list()

# 테스트 이미지 사이즈를 384로 증가.
CFG.image_size = 384

test_transform = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_dataset = BreedDataset(image_paths=test_image_paths,
                            targets=test_targets, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

trained_model = trainer.get_trained_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

[Evaluating]: 100%|██████████| 129/129 [00:52<00:00,  2.44it/s, Accuracy=0.914]

test dataset evaluation:0.9139
